In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.optimizers import SGD
from keras.utils import to_categorical,normalize
import pandas as pd
import scipy.io as scio
import numpy as np
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping


data= './icassp/Mfccwithoutdcttrainew.mat'
dataset= scio.loadmat(data)



In [ ]:
dataset['allMfcc'].shape

In [ ]:
def load_data():
    b=[]
    loc = []
    num = 0
    for i_train in range(10):        
        for j_train in range(5):
            x_box1 = dataset['allMfcc'][i_train][j_train]           
            x_box_true = np.transpose(x_box1)            
            x_box_true = x_box_true.reshape(-1)
            a = x_box_true.tolist()
            for n in range(33):
                m = (n+1)*3*32
                b = a[m-3*32:m]
                b.append(num)
                loc.append(b)                            
        num = num + 1
    traindata = np.array(loc)
    return traindata

In [ ]:
traindata = load_data()

In [ ]:
traindata.shape

In [ ]:
def load_data(traindata):
    np.random.shuffle(traindata)
    testline = int(traindata.shape[0]* (70/100))
    x_train = traindata[:testline , : -1]    
    y_train = traindata[:testline, -1]
    x_test = traindata[testline:, : -1]
    y_test = traindata[testline:, -1]
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test,10)
    return (x_train, y_train), (x_test, y_test)

In [ ]:
(x_train, y_train), (x_test, y_test) = load_data(traindata)

print(x_train.shape)
print(y_train[0])

In [ ]:
model = Sequential()

model.add(Dense(input_dim = 32 *3, units = 233))

model.add(Activation('relu'))

model.add(Dense(units=689))

model.add(Activation('relu'))

model.add(Dense(units = 689))

model.add(Activation('relu'))

model.add(Dense(units=233))
model.add(Activation('relu'))

model.add(Dense(units = 10))

model.add(Activation( 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.00001), metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(x_train, y_train, batch_size=10,epochs=20000, validation_split=0.2, callbacks=[early_stopping])
#model.fit(x_train, y_train, batch_size=1,epochs=20)

result = model.evaluate(x_train, y_train)
print("\nTrain Acc: ", result[1])

result = model.evaluate(x_test, y_test)
print("\nTest Acc: ", result[1])

classes = model.predict_classes(x_train[0:1,:])
print(classes)

In [ ]:
model.save('./my_model_32*3_withoutdct_5')